In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep
from matplotlib.colors import LogNorm
plt.style.use(mplhep.style.CMS)

In [2]:
# run_ranges_periods, df_run_ranges, fiducial_cuts, fiducial_cuts_all, aperture_period_map, aperture_parametrisation, check_aperture, get_data, process_data_protons_multiRP
from processing import *

save_figures = False

Welcome to JupyROOT 6.24/00
{'muon': {'2017B': 2.360904801, '2017C1': 5.313012839, '2017C2': 3.264135878, '2017D': 4.074723964, '2017E': 8.958810514, '2017F1': 1.708478656, '2017F2': 7.877903151, '2017F3': 3.632463163}, 'electron': {'2017B': 2.259685729466727, '2017C1': 5.070112518026598, '2017C2': 3.114906113929596, '2017D': 3.8868791892596004, '2017E': 8.568260128452684, '2017F1': 1.629432474022848, '2017F2': 7.513416205912684, '2017F3': 3.4643999898374793}}
Luminosity 2017 muon: 37.190432966
Luminosity 2017 electron: 35.507092348908216
{'muon': {'2018A': 12.1, '2018B1': 6.38, '2018B2': 0.4, '2018C': 6.5297, '2018D1': 19.88, '2018D2': 10.4157}, 'electron': {'2018A': 12.1, '2018B1': 6.38, '2018B2': 0.4, '2018C': 6.5297, '2018D1': 19.88, '2018D2': 10.4157}}
Luminosity 2018 muon: 55.7054
Luminosity 2018 electron: 55.7054


In [ ]:
help( process_data_protons_multiRP )

In [ ]:
# def get_data( fileNames ):
    
#     df_protons_multiRP_list = []
#     df_protons_singleRP_list = []
#     df_ppstracks_list = []
#     df_counts_list = []

#     for file_ in fileNames:
#         print ( file_ )
#         with h5py.File( file_, 'r' ) as f:
#             print ( list(f.keys()) )

#             dset_protons_multiRP = f['protons_multiRP']
#             print ( dset_protons_multiRP.shape )
#             print ( dset_protons_multiRP[:,:] )

#             dset_protons_singleRP = f['protons_singleRP']
#             print ( dset_protons_singleRP.shape )
#             print ( dset_protons_singleRP[:,:] )

#             dset_ppstracks = f['ppstracks']
#             print ( dset_ppstracks.shape )
#             print ( dset_ppstracks[:,:] )

#             dset_columns_protons = f['columns_protons']
#             print ( dset_columns_protons.shape )
#             columns_protons = list( dset_columns_protons )
#             print ( columns_protons )
#             columns_protons_str = [ item.decode("utf-8") for item in columns_protons ]
#             print ( columns_protons_str )

#             dset_columns_ppstracks = f['columns_ppstracks']
#             print ( dset_columns_ppstracks.shape )
#             columns_ppstracks = list( dset_columns_ppstracks )
#             print ( columns_ppstracks )
#             columns_ppstracks_str = [ item.decode("utf-8") for item in columns_ppstracks ]
#             print ( columns_ppstracks_str )

#             dset_selections = f['selections']
#             selections_ = [ item.decode("utf-8") for item in dset_selections ]
#             print ( selections_ )

#             dset_counts = f['event_counts']
#             df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
#             print ( df_counts_list[-1] )

#             chunk_size = 1000000
#             entries_protons_multiRP = dset_protons_multiRP.shape[0]
#             start_ = list( range( 0, entries_protons_multiRP, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_protons_multiRP )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_protons_multiRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
#                                         { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
#                                           "muon0_charge": "int32",
#                                           "nVertices": "int32",
#                                           "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
#                                           "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
#                 df_protons_multiRP_list.append( df_ )
#                 print ( df_protons_multiRP_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_protons_multiRP_list[-1] ) ) )

#             entries_protons_singleRP = dset_protons_singleRP.shape[0]
#             start_ = list( range( 0, entries_protons_singleRP, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_protons_singleRP )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_protons_singleRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
#                                         { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
#                                           "muon0_charge": "int32",
#                                           "nVertices": "int32",
#                                           "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
#                                           "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
#                 df_protons_singleRP_list.append( df_ )
#                 print ( df_protons_singleRP_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_protons_singleRP_list[-1] ) ) )

#             entries_ppstracks = dset_ppstracks.shape[0]
#             start_ = list( range( 0, entries_ppstracks, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_ppstracks )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_ppstracks[ start_[idx] : stop_[idx] ], columns=columns_ppstracks_str ).astype( { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "rpid": "int32" } )
#                 df_ppstracks_list.append( df_ )
#                 print ( df_ppstracks_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_ppstracks_list[-1] ) ) )

#     df_counts = df_counts_list[0]
#     for idx in range( 1, len( df_counts_list ) ):
#         df_counts = df_counts.add( df_counts_list[idx] )
#     print ( df_counts )

#     df_protons_multiRP = pd.concat( df_protons_multiRP_list )
#     print (df_protons_multiRP)

#     df_protons_singleRP = pd.concat( df_protons_singleRP_list )
#     print (df_protons_singleRP)

#     df_ppstracks = pd.concat( df_ppstracks_list )
#     print (df_ppstracks)
    
#     return (df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks)

In [ ]:
# run_ranges_periods = {}
# run_ranges_periods[ "2017B" ]  = (297020,299329)
# run_ranges_periods[ "2017C1" ] = (299337,300785)
# run_ranges_periods[ "2017C2" ] = (300806,302029)
# run_ranges_periods[ "2017D" ]  = (302030,303434)
# run_ranges_periods[ "2017E" ]  = (303435,304826)
# run_ranges_periods[ "2017F1" ] = (304911,305114)
# run_ranges_periods[ "2017F2" ] = (305178,305902)
# run_ranges_periods[ "2017F3" ] = (305965,306462)
# df_run_ranges = pd.DataFrame( run_ranges_periods, index=("min","max") ).transpose()

# aperture_period_map = {
#     "2016_preTS2"  : "2016_preTS2",
#     "2016_postTS2" : "2016_postTS2",
#     "2017B"        : "2017_preTS2",
#     "2017C1"       : "2017_preTS2",
#     "2017C2"       : "2017_preTS2",
#     "2017D"        : "2017_preTS2",
#     "2017E"        : "2017_postTS2",
#     "2017F1"       : "2017_postTS2",
#     "2017F2"       : "2017_postTS2",
#     "2017F3"       : "2017_postTS2",
#     "2018"         : "2018"
# }

In [ ]:
# # Per data period, arm=(0,1)
# # Periods: "2016_preTS2", "2016_postTS2", "2017_preTS2", "2017_postTS2", "2018"
# def aperture_parametrisation( period, arm, xangle, xi ):

#     #https://github.com/cms-sw/cmssw/tree/916cb3d20213734a0465240720c8c8c392b92eac/Validation/CTPPS/python/simu_config

#     if (period == "2016_preTS2"):
#         if   (arm == 0): return 3.76296E-05+((xi<0.117122)*0.00712775+(xi>=0.117122)*0.0148651)*(xi-0.117122);
#         elif (arm == 1): return 1.85954E-05+((xi<0.14324)*0.00475349+(xi>=0.14324)*0.00629514)*(xi-0.14324);
#     elif (period == "2016_postTS2"):
#         if   (arm == 0): return 6.10374E-05+((xi<0.113491)*0.00795942+(xi>=0.113491)*0.01935)*(xi-0.113491);
#         elif (arm == 1): return (xi-0.110)/130.0;
#     elif (period == "2017_preTS2"):
#         if   (arm == 0): return -(8.71198E-07*xangle-0.000134726)+((xi<(0.000264704*xangle+0.081951))*-(4.32065E-05*xangle-0.0130746)+(xi>=(0.000264704*xangle+0.081951))*-(0.000183472*xangle-0.0395241))*(xi-(0.000264704*xangle+0.081951));
#         elif (arm == 1): return 3.43116E-05+((xi<(0.000626936*xangle+0.061324))*0.00654394+(xi>=(0.000626936*xangle+0.061324))*-(0.000145164*xangle-0.0272919))*(xi-(0.000626936*xangle+0.061324));
#     elif (period == "2017_postTS2"):
#         if   (arm == 0): return -(8.92079E-07*xangle-0.000150214)+((xi<(0.000278622*xangle+0.0964383))*-(3.9541e-05*xangle-0.0115104)+(xi>=(0.000278622*xangle+0.0964383))*-(0.000108249*xangle-0.0249303))*(xi-(0.000278622*xangle+0.0964383));
#         elif (arm == 1): return 4.56961E-05+((xi<(0.00075625*xangle+0.0643361))*-(3.01107e-05*xangle-0.00985126)+(xi>=(0.00075625*xangle+0.0643361))*-(8.95437e-05*xangle-0.0169474))*(xi-(0.00075625*xangle+0.0643361));
#     elif (period == "2018"):
#         if   (arm == 0): return -(8.44219E-07*xangle-0.000100957)+((xi<(0.000247185*xangle+0.101599))*-(1.40289E-05*xangle-0.00727237)+(xi>=(0.000247185*xangle+0.101599))*-(0.000107811*xangle-0.0261867))*(xi-(0.000247185*xangle+0.101599));
#         elif (arm == 1): return -(-4.74758E-07*xangle+3.0881E-05)+((xi<(0.000727859*xangle+0.0722653))*-(2.43968E-05*xangle-0.0085461)+(xi>=(0.000727859*xangle+0.0722653))*-(7.19216E-05*xangle-0.0148267))*(xi-(0.000727859*xangle+0.0722653));
#     else:
#         return -999.

# def check_aperture( period, arm, xangle, xi, theta_x ):
#     return ( theta_x < -aperture_parametrisation( period, arm, xangle, xi ) )

In [ ]:
# def process_data_protons_multiRP( df_protons_multiRP, df_ppstracks=None, runOnMC=False ):

#     if not runOnMC:
#         df_protons_multiRP.loc[ :, "period" ] = np.nan
#         for idx_ in range( df_run_ranges.shape[0] ):
#             msk_period_ = ( ( df_protons_multiRP.loc[ :, "run" ] >= df_run_ranges.iloc[ idx_ ][ "min" ] ) & ( df_protons_multiRP.loc[ :, "run" ] <= df_run_ranges.iloc[ idx_ ][ "max" ] ) )
#             sum_period_ = np.sum( msk_period_ )
#             if sum_period_ > 0:
#                 period_key_ = df_run_ranges.index[ idx_ ]
#                 df_protons_multiRP.loc[ :, "period" ].loc[ msk_period_ ] = period_key_
#                 print ( "{}: {}".format( period_key_, sum_period_ ) )

#         df_protons_multiRP.loc[ :, "within_aperture" ] = df_protons_multiRP.apply(
#                 lambda row: check_aperture( aperture_period_map[ row["period"] ], row["arm"], 120., row["xi"], row["thx"] ), # FIXME
#                 axis=1
#                 )
    
#     df_protons_multiRP_index = df_protons_multiRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )

#     df_ppstracks_index = None
#     if not df_ppstracks is None:
#         df_ppstracks_index = df_ppstracks.set_index( ['run', 'lumiblock', 'event', 'slice'] )
        
#         df_protons_multiRP_index.loc[ :, "track_x1" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_x2" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_y1" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_y2" ] = np.nan
        
#         df_protons_multiRP_index.loc[ :, "track_x1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 3 ]
#         df_protons_multiRP_index.loc[ :, "track_x2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 23 ]
#         df_protons_multiRP_index.loc[ :, "track_y1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 3 ]
#         df_protons_multiRP_index.loc[ :, "track_y2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 23 ]
#         df_protons_multiRP_index.loc[ :, "track_x1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 103 ]
#         df_protons_multiRP_index.loc[ :, "track_x2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 123 ]
#         df_protons_multiRP_index.loc[ :, "track_y1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 103 ]
#         df_protons_multiRP_index.loc[ :, "track_y2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 123 ]

#     df_protons_multiRP_events = df_protons_multiRP_index.drop( columns=[ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm" ] )
#     df_protons_multiRP_events = df_protons_multiRP_events[ ~df_protons_multiRP_events.index.duplicated(keep='first') ]

#     df_protons_multiRP_events.loc[ :, "MX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 13000. * np.sqrt( df_.iloc[0].xi * df_.iloc[1].xi ) )
#     df_protons_multiRP_events.loc[ :, "YX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 0.5 * np.log( df_.iloc[0].xi / df_.iloc[1].xi ) )
#     df_protons_multiRP_events.loc[ :, "diffMWW_MX" ]  = df_protons_multiRP_events[ "recoMWW" ] - df_protons_multiRP_events[ "MX" ]
#     df_protons_multiRP_events.loc[ :, "ratioMWW_MX" ] = df_protons_multiRP_events[ "recoMWW" ] / df_protons_multiRP_events[ "MX" ]
#     df_protons_multiRP_events.loc[ :, "shiftedRatioMWW_MX" ] = df_protons_multiRP_events[ "ratioMWW_MX" ] - 1.
#     df_protons_multiRP_events.loc[ :, "diffYWW_YX" ]  = df_protons_multiRP_events[ "recoRapidityWW" ] - df_protons_multiRP_events[ "YX" ]
    
#     return (df_protons_multiRP_index, df_protons_multiRP_events, df_ppstracks_index)

### Signal

In [ ]:
# run_tables = False

# # labels_signals = [ "GGToWW-AQGC-A0W1e-6" ]
# labels_signals = [ "GGToWW-AQGC-A0W1e-6", "GGToWW-AQGC-A0W2e-6", "GGToWW-AQGC-A0W5e-6" ]

# fileNames_signals = {
#     "GGToWW-AQGC-A0W1e-6": [ "output-GGToWW-AQGC-A0W1e-6.h5" ],
#     "GGToWW-AQGC-A0W2e-6": [ "output-GGToWW-AQGC-A0W2e-6.h5" ],
#     "GGToWW-AQGC-A0W5e-6": [ "output-GGToWW-AQGC-A0W5e-6.h5" ]
#     }

# df_counts_signals = {}
# df_signals_protons_multiRP_index = {}
# df_signals_protons_multiRP_events = {}

# for label_ in labels_signals:
#     import time
#     print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
#     time_s_ = time.time()

#     #df_counts_signal, df_protons_multiRP_signal, df_protons_singleRP_signal, df_ppstracks_signal = 4 * [None]
#     #df_protons_multiRP_signal_index, df_protons_multiRP_signal_events, df_ppstracks_signal_index = 3 * [None]

#     if run_tables:
#         with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), complevel=5 ) as store_:

#             df_counts_signal_, df_protons_multiRP_signal_, df_protons_singleRP_signal_, df_ppstracks_signal_ = get_data( fileNames_signals[ label_ ] )
#             df_protons_multiRP_signal_index_, df_protons_multiRP_signal_events_, df_ppstracks_signal_index_ = process_data_protons_multiRP( df_protons_multiRP_signal_, df_ppstracks_signal_, apply_fiducial=True, within_aperture=True, select_2protons=False, runOnMC=True )

#             store_[ "counts" ] = df_counts_signal_
#             store_[ "protons_multiRP"] = df_protons_multiRP_signal_index_
#             store_[ "events_multiRP" ] = df_protons_multiRP_signal_events_
    
#     with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), 'r' ) as store_:
#         df_counts_signals[ label_ ] = store_[ "counts" ]
#         df_signals_protons_multiRP_index[ label_ ] = store_[ "protons_multiRP" ]
#         df_signals_protons_multiRP_events[ label_ ] = store_[ "events_multiRP" ]

#     time_e_ = time.time()
#     print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
# for label_ in labels_signals:
#     with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), 'r' ) as store_:
#         print ( list( store_ ) )

In [26]:
# labels_signals = [ "GGToWW-AQGC-A0W1e-6", "GGToWW-AQGC-A0W2e-6", "GGToWW-AQGC-A0W5e-6" ]
labels_signals = [ "GGToWW-AQGC-2018-muon-A0W1e-6", "GGToWW-AQGC-2018-muon-A0W2e-6", "GGToWW-AQGC-2018-muon-A0W5e-6" ]
df_counts_signals = {}
df_signals_protons_multiRP_index = {}
df_signals_protons_multiRP_events = {}
base_path_ = 'output'
# base_path_ = 'output/save-V5'
for label_ in labels_signals:
    print ( label_ )
    file_path_ = "{}/data-store-{}.h5".format( base_path_, label_ )
    with pd.HDFStore( file_path_, 'r' ) as store_:
        print ( list( store_ ) )
        df_counts_signals[ label_ ] = store_[ "counts" ]
        df_signals_protons_multiRP_index[ label_ ] = store_[ "protons_multiRP" ]
        df_signals_protons_multiRP_events[ label_ ] = store_[ "events_multiRP" ]

GGToWW-AQGC-2018-muon-A0W1e-6
['/counts', '/events_multiRP', '/protons_multiRP']
GGToWW-AQGC-2018-muon-A0W2e-6
['/counts', '/events_multiRP', '/protons_multiRP']
GGToWW-AQGC-2018-muon-A0W5e-6
['/counts', '/events_multiRP', '/protons_multiRP']


In [29]:
# label_ = "GGToWW-AQGC-A0W1e-6"
label_ = "GGToWW-AQGC-2018-muon-A0W1e-6"
df_counts_signals[ label_ ]

All       4810
Jet       4810
Muon      4810
Proton    3517
dtype: int64

In [30]:
df_signals_protons_multiRP_index[ label_ ].columns.values

array(['crossingAngle', 'betaStar', 'instLumi', 'xi', 'thx', 'thy', 't',
       'ismultirp', 'rpid', 'arm', 'random', 'jet0_pt', 'jet0_eta',
       'jet0_phi', 'jet0_energy', 'jet0_mass', 'jet0_corrmass',
       'jet0_tau1', 'jet0_tau2', 'jet0_vertexz', 'jet0_px', 'jet0_py',
       'jet0_pz', 'met', 'met_x', 'met_y', 'met_phi', 'nVertices',
       'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4',
       'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad',
       'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt',
       'WLeptonicEta', 'WLeptonicPhi', 'muon0_pt', 'muon0_eta',
       'muon0_phi', 'muon0_energy', 'muon0_charge', 'muon0_iso',
       'muon0_dxy', 'muon0_dz', 'run_mc', 'pileupWeight',
       'mc_pu_trueinteractions', 'mcWeight', 'jet0_jer_res',
       'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'gen_jet0_pt',
       'gen_jet0_eta', 'gen_jet0_phi', 'gen_jet0_energy', 'trackx1',
       'tracky1', 'trackpixshift1', 'rpid1', 'trackx2', 'tracky2',
   

In [32]:
df_signals_protons_multiRP_index[ label_ ]
# df_signals_protons_multiRP_index[ label_ ][ ['pileupWeight','mc_pu_trueinteractions'] ]

crossingAngle  betaStar  \
run lumiblock event hash_id              slice                            
1   1         1     11792644136667482086 0              155.0       0.4   
                                         0              155.0       0.4   
                    6050478515959135830  0              155.0       0.4   
                    3655724426672568106  0              155.0       0.4   
                    13726604837472925229 0              155.0       0.4   
...                                                       ...       ...   
    995       995   12882894790533776945 0              160.0       0.4   
                    8428429390990631370  0              160.0       0.4   
                    6668014878596475468  0              160.0       0.4   
    996       996   15366430552113981237 0              145.0       0.4   
    998       998   12932831343316490139 0              160.0       0.4   

                                                instLumi        xi       thx  \
run lumiblock event hash_id              slice                                 
1   1         1     11792644136667482086 0           0.0  0.061111  0.000116   
                                         0           0.0  0.066139  0.000032   
                    6050478515959135830  0           0.0  0.137584 -0.000082   
                    3655724426672568106  0           0.0  0.080872  0.000042   
                    13726604837472925229 0           0.0  0.111399 -0.000038   
...                                                  ...       ...       ...   
    995       995   12882894790533776945 0           0.0  0.069416 -0.000021   
                    8428429390990631370  0           0.0  0.072316  0.000008   
                    6668014878596475468  0           0.0  0.139108  0.000038   
    996       996   15366430552113981237 0           0.0  0.062790  0.000050   
    998       998   12932831343316490139 0           0.0  0.094851 -0.000082   

                                                     thy         t  ismultirp  \
run lumiblock event hash_id              slice                                  
1   1         1     11792644136667482086 0     -0.000046 -0.619278          1   
                                         0      0.000050 -0.145626          1   
                    6050478515959135830  0      0.000023 -0.280269          1   
                    3655724426672568106  0      0.000042 -0.142402          1   
                    13726604837472925229 0     -0.000032 -0.102597          1   
...                                                  ...       ...        ...   
    995       995   12882894790533776945 0     -0.000009 -0.024684          1   
                    8428429390990631370  0     -0.000045 -0.087506          1   
                    6668014878596475468  0     -0.000004 -0.072704          1   
    996       996   15366430552113981237 0     -0.000002 -0.104547          1   
    998       998   12932831343316490139 0      0.000133 -0.941141          1   

                                                rpid  arm  ...  yhigh  \
run lumiblock event hash_id              slice             ...          
1   1         1     11792644136667482086 0       123    0  ...  3.298   
                                         0       123    1  ...  3.898   
                    6050478515959135830  0       123    0  ...  3.298   
                    3655724426672568106  0        23    0  ...  3.298   
                    13726604837472925229 0       123    1  ...  3.898   
...                                              ...  ...  ...    ...   
    995       995   12882894790533776945 0        23    0  ...  3.298   
                    8428429390990631370  0       103    0  ...  3.298   
                    6668014878596475468  0        23    0  ...  3.298   
    996       996   15366430552113981237 0        23    0  ...  3.298   
    998       998   12932831343316490139 0        23    0  ...  3.298   

                           

In [23]:
df_signals_protons_multiRP_index[ label_ ][ [ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm", "random", "trackpixshift1", "rpid1", "trackpixshift2", "rpid2", "trackx1", "trackx2", "tracky1", "tracky2" ] ]

xi       thx       thy         t  ismultirp  \
run lumiblock event  slice                                                      
1   2266      226549 0      0.049396  0.000070  0.000006 -0.202391          1   
              226551 0      0.047449  0.000007  0.000021 -0.022129          1   
                     0      0.051457  0.000022  0.000089 -0.342805          1   
    2267      226605 0      0.129090  0.000042 -0.000010 -0.086362          1   
              226690 0      0.028568  0.000061 -0.000014 -0.162398          1   
...                              ...       ...       ...       ...        ...   
    9051      405030 0      0.107774 -0.000052  0.000093 -0.437632          1   
    9052      405137 0      0.109196 -0.000054  0.000128 -0.733809          1   
    9057      405631 0      0.098174 -0.000035 -0.000023 -0.076592          1   
              405635 0      0.119186  0.000044  0.000081 -0.331256          1   
              405671 0      0.077515  0.000018  0.000056 -0.140505          1   

                            rpid  arm  random  trackpixshift1  rpid1  \
run lumiblock event  slice                                             
1   2266      226549 0       123    1       0               0    103   
              226551 0       123    0       0               0      3   
                     0       123    1       0               0    103   
    2267      226605 0       123    1       0               0    103   
              226690 0        23    0       0               0      3   
...                          ...  ...     ...             ...    ...   
    9051      405030 0       103    0       0               0      3   
    9052      405137 0        23    0       0               0      3   
    9057      405631 0       123    1       0               0    103   
              405635 0       123    1       0               0    103   
              405671 0       123    1       0               0    103   

                            trackpixshift2  rpid2    trackx1    trackx2  \
run lumiblock event  slice                                                
1   2266      226549 0                   0    123   4.863037   4.985718   
              226551 0                   0     23   5.332520   5.407166   
                     0                   0    123   4.510742   4.530273   
    2267      226605 0                   0    123  13.440430  13.487671   
              226690 0                   0     23   3.356812   3.509430   
...                                    ...    ...        ...        ...   
    9051      405030 0                   0     23   9.653320   9.582275   
    9052      405137 0                   0     23  10.827148  10.796753   
    9057      405631 0                   0    123   7.567627   7.414856   
              405635 0                   0    123  12.294434  12.348999   
              405671 0                   0    123   6.961670   6.959412   

                             tracky1   tracky2  
run lumiblock event  slice                      
1   2266      226549 0      0.647461  0.700012  
              226551 0      0.349609  0.299988  
                     0     -0.235474 -0.350006  
    2267      226605 0      3.787598  4.049805  
              226690 0      0.429138  0.450012  
...                              ...       ...  
    9051      405030 0     -2.228027 -2.500000  
    9052      405137 0     -2.955811 -3.300049  
    9057      405631 0      2.158691  2.350098  
              405635 0     -1.612061 -1.699951  
              405671 0     -0.307739 -0.350006  

[14577 rows x 16 columns]

In [16]:
msk_columns_jet_ = [ key_[:len('jet')] == 'jet' for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_jet_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_jet_ ]
columns_ = [ "period" ]
columns_.extend( columns_jet_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

period     jet0_pt  jet0_eta  \
run lumiblock event hash_id              slice                                 
1   1         1     11792644136667482086 0       2018C  324.683563 -0.696189   
                                         0       2018C  324.683563 -0.696189   
                    6050478515959135830  0      2018D2  240.310043  1.931237   
                    3655724426672568106  0      2018D1  260.017914  0.650234   
                    13726604837472925229 0      2018D1  228.105728  0.204343   
...                                                ...         ...       ...   
    995       995   12882894790533776945 0      2018D2  216.897644  0.456907   
                    8428429390990631370  0       2018A  609.528137  0.380333   
                    6668014878596475468  0      2018D2  318.403717  0.609904   
    996       996   15366430552113981237 0      2018B1  200.760803  0.922690   
    998       998   12932831343316490139 0      2018D2  261.364288  1.256408   

                                                jet0_phi  jet0_energy  \
run lumiblock event hash_id              slice                          
1   1         1     11792644136667482086 0      2.297015   422.007355   
                                         0      2.297015   422.007355   
                    6050478515959135830  0      1.718068   850.382629   
                    3655724426672568106  0     -2.584588   320.355957   
                    13726604837472925229 0      0.157712   236.944717   
...                                                  ...          ...   
    995       995   12882894790533776945 0     -1.567587   252.918304   
                    8428429390990631370  0     -1.424535   662.385620   
                    6668014878596475468  0      1.448195   387.606903   
    996       996   15366430552113981237 0     -0.753148   295.341614   
    998       998   12932831343316490139 0      0.999958   503.750854   

                                                jet0_mass  jet0_corrmass  \
run lumiblock event hash_id              slice                             
1   1         1     11792644136667482086 0      95.735313     100.935997   
                                         0      95.735313     100.935997   
                    6050478515959135830  0      68.957008      73.930748   
                    3655724426672568106  0       2.295485       2.406761   
                    13726604837472925229 0       8.800271       9.234779   
...                                                   ...            ...   
    995       995   12882894790533776945 0      82.655945      86.896072   
                    8428429390990631370  0      84.276901      88.628273   
                    6668014878596475468  0      73.266174      77.003136   
    996       996   15366430552113981237 0      15.779487      17.323191   
    998       998   12932831343316490139 0      67.330406      74.306694   

                                                jet0_tau1  jet0_tau2  \
run lumiblock event hash_id              slice                         
1   1         1     11792644136667482086 0       0.318949   0.112376   
                                         0       0.318949   0.112376   
                    6050478515959135830  0       0.318457   0.095690   
                    3655724426672568106  0       0.073669   0.059655   
                    13726604837472925229 0       0.101172   0.082598   
...                                                   ...        ...   
    995       995   12882894790533776945 0       0.414411   0.198345   
                    8428429390990631370  0       0.199205   0.024355   
                    6668014878596475468  0       0.287917   0.065268   
    996       996   15366430552113981237 0       0.134714   0.106617   
    998       998   12932831343316490139 0       0.350170   0.076654   

                                                jet0_vertexz  ...  \
run lumiblock event hash_id              slice                

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.hist( df_signals_protons_multiRP_index[ label_ ][ "jet0_eta" ], bins=50, range=(-5.,5.) )

In [ ]:
# df__ = df_signals_protons_multiRP_index[ label_ ]
# df__.loc[ :, 'jet0_pt_unsmeared' ] = np.sqrt(
#     df__.loc[ :, 'jet0_px' ] ** 2 +
#     df__.loc[ :, 'jet0_py' ] ** 2 )
# df__.loc[ :, 'C_JER_ref' ] = ( df__.loc[ :, 'jet0_pt' ] / df__.loc[ :, 'jet0_pt_unsmeared' ] )
# deltaPhi_jet_genjet_ = ( df__.loc[ :, "jet0_phi"] - df__.loc[ :, "gen_jet0_phi"] )
# msk_dphi_ = ( deltaPhi_jet_genjet_ < -np.pi )
# deltaPhi_jet_genjet_.loc[ msk_dphi_ ] = deltaPhi_jet_genjet_.loc[ msk_dphi_ ] + 2*np.pi
# msk_dphi_ = ( deltaPhi_jet_genjet_ >= np.pi )
# deltaPhi_jet_genjet_.loc[ msk_dphi_ ] = deltaPhi_jet_genjet_.loc[ msk_dphi_ ] - 2*np.pi
# deltaEta_jet_genjet_ = ( df__.loc[ :, "jet0_eta"] - df__.loc[ :, "gen_jet0_eta"] )
# deltaR_jet_genjet_ = np.sqrt( ( deltaPhi_jet_genjet_ ) ** 2 + ( deltaEta_jet_genjet_ ) ** 2 )
# df__.loc[ :, 'deltaR_jet_genjet' ] = deltaR_jet_genjet_
# df__.loc[ :, 'deltaPt_jet_genjet' ] = np.abs( df__.loc[ :, 'jet0_pt_unsmeared' ] - df__.loc[ :, 'gen_jet0_pt' ] )
# df__.loc[ :, 'match_jet_genjet' ] = ( ( df__.loc[ :, 'deltaR_jet_genjet' ] < ( 0.8 / 2 ) ) &
#                                       ( df__.loc[ :, 'deltaPt_jet_genjet' ] < ( 3. * df__.loc[ :, 'jet0_jer_res' ] * df__.loc[ :, 'jet0_pt_unsmeared' ] ) ) )  
# print ( np.sum( df__.loc[ :, 'match_jet_genjet' ] ) )
# print ( np.sum( df__.loc[ :, 'match_jet_genjet' ] ) / df__.shape[0] )

# # df__[ [ 'C_JER_ref', 'jet0_jer_res', 'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'jet0_pt_unsmeared', 'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_px', 'jet0_py', 'jet0_pt_nom', 'jet0_px_nom', 'jet0_py_nom', 'gen_jet0_pt', 'gen_jet0_eta', 'gen_jet0_phi', 'deltaR_jet_genjet', 'deltaPt_jet_genjet', 'match_jet_genjet' ] ]
# df__[ [ 'C_JER_ref', 'jet0_jer_res', 'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'jet0_pt_unsmeared', 'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_px', 'jet0_py', 'jet0_pt_nom', 'jet0_px_nom', 'jet0_py_nom', 'gen_jet0_pt', 'gen_jet0_eta', 'gen_jet0_phi', 'deltaR_jet_genjet', 'deltaPt_jet_genjet', 'match_jet_genjet' ] ].loc[ ~df__.loc[ :, 'match_jet_genjet' ] ]

In [ ]:
# df__.loc[ :, 'C_JER' ] = np.nan
# df__.loc[ :, 'JER_rand' ] = np.nan
# df__.loc[ :, 'C_JER_jer_up' ] = np.nan
# df__.loc[ :, 'C_JER_jer_dw' ] = np.nan
# msk_match_ = df__.loc[ :, 'match_jet_genjet' ]

# df__.loc[ :, 'C_JER' ].where( ~msk_match_, ( 1. + ( df__.loc[ :, 'jet0_jer_sf' ] - 1. ) *
#                                            ( ( df__.loc[ :, 'jet0_pt_unsmeared' ] - df__.loc[ :, 'gen_jet0_pt' ] ) / df__.loc[ :, 'jet0_pt_unsmeared' ] ) ),
#                                            inplace=True )
# df__.loc[ :, 'C_JER' ].where( msk_match_, df__.loc[ :, 'C_JER_ref' ], inplace=True )
# df__.loc[ :, 'JER_rand' ].where( msk_match_, ( ( df__.loc[ :, 'C_JER_ref' ] - 1. ) / np.sqrt( np.max( ( ( df__.loc[ :, 'jet0_jer_sf' ] ** 2 ) - 1. ) , 0. ) ) ), inplace=True )
# df__.loc[ :, 'C_JER_jer_up' ].where( ~msk_match_, ( 1. + ( df__.loc[ :, 'jet0_jer_sfup' ] - 1. ) *
#                                                   ( ( df__.loc[ :, 'jet0_pt_unsmeared' ] - df__.loc[ :, 'gen_jet0_pt' ] ) / df__.loc[ :, 'jet0_pt_unsmeared' ] ) ),
#                                                   inplace=True )
# df__.loc[ :, 'C_JER_jer_dw' ].where( ~msk_match_, ( 1. + ( df__.loc[ :, 'jet0_jer_sfdown' ] - 1. ) *
#                                                   ( ( df__.loc[ :, 'jet0_pt_unsmeared' ] - df__.loc[ :, 'gen_jet0_pt' ] ) / df__.loc[ :, 'jet0_pt_unsmeared' ] ) ),
#                                                   inplace=True )
# df__.loc[ :, 'C_JER_jer_up' ].where( msk_match_, ( 1. + df__.loc[ :, 'JER_rand' ] * np.sqrt( np.max( ( ( df__.loc[ :, 'jet0_jer_sfup' ] ** 2 ) - 1. ) , 0. ) ) ), inplace=True )
# df__.loc[ :, 'C_JER_jer_dw' ].where( msk_match_, ( 1. + df__.loc[ :, 'JER_rand' ] * np.sqrt( np.max( ( ( df__.loc[ :, 'jet0_jer_sfdown' ] ** 2 ) - 1. ) , 0. ) ) ), inplace=True )


In [ ]:
df__ = df_signals_protons_multiRP_index[ label_ ]

# df__[ [ 'C_JER_ref', 'C_JER', 'JER_rand', 'C_JER_jer_up', 'C_JER_jer_dw', 'jet0_jer_res', 'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'jet0_pt_unsmeared', 'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_px', 'jet0_py', 'jet0_pt_nom', 'jet0_px_nom', 'jet0_py_nom', 'gen_jet0_pt', 'gen_jet0_eta', 'gen_jet0_phi', 'deltaR_jet_genjet', 'deltaPt_jet_genjet', 'match_jet_genjet' ] ]
df__[ [ 'C_JER_ref', 'C_JER', 'JER_rand', 'C_JER_jer_up', 'C_JER_jer_dw', 'jet0_jer_res', 'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'jet0_pt_unsmeared', 'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_px', 'jet0_py', 'jet0_pt_nom', 'jet0_px_nom', 'jet0_py_nom', 'gen_jet0_pt', 'gen_jet0_eta', 'gen_jet0_phi', 'deltaR_jet_genjet', 'deltaPt_jet_genjet', 'match_jet_genjet' ] ].loc[ df__.loc[ :, 'match_jet_genjet' ] ]
# df__[ [ 'C_JER_ref', 'C_JER', 'JER_rand', 'C_JER_jer_up', 'C_JER_jer_dw', 'jet0_jer_res', 'jet0_jer_sf', 'jet0_jer_sfup', 'jet0_jer_sfdown', 'jet0_pt_unsmeared', 'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_px', 'jet0_py', 'jet0_pt_nom', 'jet0_px_nom', 'jet0_py_nom', 'gen_jet0_pt', 'gen_jet0_eta', 'gen_jet0_phi', 'deltaR_jet_genjet', 'deltaPt_jet_genjet', 'match_jet_genjet' ] ].loc[ ~df__.loc[ :, 'match_jet_genjet' ] ]

In [ ]:
msk_match_ = df__.loc[ :, 'match_jet_genjet' ]
print ( np.mean( df__.loc[ msk_match_ ].loc[ :, 'C_JER' ] - df__.loc[ msk_match_ ].loc[ :, 'C_JER_ref' ] ) ) 
print ( np.std( df__.loc[ msk_match_ ].loc[ :, 'C_JER' ] - df__.loc[ msk_match_ ].loc[ :, 'C_JER_ref' ] ) )
print ( np.mean( df__.loc[ ~msk_match_ ].loc[ :, 'JER_rand' ] ) )
print ( np.std( df__.loc[ ~msk_match_ ].loc[ :, 'JER_rand' ] ) )

In [ ]:
fig = plt.figure( figsize=(12,10) )
# plt.hist( df__.loc[ ~msk_match_ ].loc[ :, 'JER_rand' ], bins=50, range=(-0.2, 0.2) )
# plt.hist( ( df__.loc[ msk_match_ ].loc[ :, 'C_JER' ] - df__.loc[ msk_match_ ].loc[ :, 'C_JER_ref' ] ), bins=50, range=(-0.05, 0.05) )
plt.hist( ( df__.loc[ ~msk_match_ ].loc[ :, 'C_JER_ref' ] ), bins=50, range=(0.90, 1.10), histtype='step', label='C_JER' )
plt.hist( ( df__.loc[ ~msk_match_ ].loc[ :, 'C_JER_jer_up' ] ), bins=50, range=(0.90, 1.10), histtype='step', label='C_JER_jer_up' )
plt.hist( ( df__.loc[ ~msk_match_ ].loc[ :, 'C_JER_jer_dw' ] ), bins=50, range=(0.90, 1.10), histtype='step', label='C_JER_jer_dw' )
plt.legend( loc='best', fontsize=16 )
# plt.hist( ( df__.loc[ msk_match_ ].loc[ :, 'C_JER_ref' ] ), bins=50, range=(0.90, 1.10), histtype='step', label='C_JER' )
# plt.hist( ( df__.loc[ msk_match_ ].loc[ :, 'C_JER_jer_up' ] / df__.loc[ :, 'C_JER' ] ), bins=50, range=(0.90, 1.10), histtype='step', label='C_JER_jer_up' )
# plt.hist( ( df__.loc[ msk_match_ ].loc[ :, 'C_JER_jer_dw' ] / df__.loc[ :, 'C_JER' ] ), bins=100, range=(0.90, 1.10), histtype='step', label='C_JER_jer_dw' )
# plt.legend( loc='best', fontsize=16 )

In [ ]:
msk_columns_jes_ = [ key_.find( 'jes' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_jes_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_jes_ ]
columns_ = [ "period" ]
columns_.extend( columns_jes_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_jer_ = [ key_.find( 'jer' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_jer_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_jer_ ]
columns_ = [ "period" ]
columns_.extend( columns_jer_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_WLep_ = [ key_.find( 'WLep') >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_WLep_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_WLep_ ]
columns_ = [ "period", "recoMWlep" ]
columns_.extend( columns_WLep_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_WW_ = [ key_.find( 'WW') >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_WW_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_WW_ ]
columns_ = [ "period" ]
columns_.extend( columns_WW_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [15]:
msk_columns_xi_ = [ key_.find( 'xi' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_xi_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_xi_ ]
columns_ = [ "period" ]
columns_.extend( columns_xi_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

period        xi  \
run lumiblock event hash_id              slice                     
1   1         1     11792644136667482086 0       2018C  0.061111   
                                         0       2018C  0.066139   
                    6050478515959135830  0      2018D2  0.137584   
                    3655724426672568106  0      2018D1  0.080872   
                    13726604837472925229 0      2018D1  0.111399   
...                                                ...       ...   
    995       995   12882894790533776945 0      2018D2  0.069416   
                    8428429390990631370  0       2018A  0.072316   
                    6668014878596475468  0      2018D2  0.139108   
    996       996   15366430552113981237 0      2018B1  0.062790   
    998       998   12932831343316490139 0      2018D2  0.094851   

                                                xiCMS_45_nom  xiCMS_56_nom  \
run lumiblock event hash_id              slice                               
1   1         1     11792644136667482086 0          0.046848      0.066613   
                                         0          0.046848      0.066613   
                    6050478515959135830  0          0.141543      0.023303   
                    3655724426672568106  0          0.087765      0.019407   
                    13726604837472925229 0          0.027860      0.065287   
...                                                      ...           ...   
    995       995   12882894790533776945 0          0.063559      0.018575   
                    8428429390990631370  0          0.088288      0.140647   
                    6668014878596475468  0          0.107229      0.024674   
    996       996   15366430552113981237 0          0.064369      0.018282   
    998       998   12932831343316490139 0          0.086745      0.027215   

                                                xiCMS_45_jes_up  \
run lumiblock event hash_id              slice                    
1   1         1     11792644136667482086 0             0.046920   
                                         0             0.046920   
                    6050478515959135830  0             0.142632   
                    3655724426672568106  0             0.088002   
                    13726604837472925229 0             0.027991   
...                                                         ...   
    995       995   12882894790533776945 0             0.063723   
                    8428429390990631370  0             0.088594   
                    6668014878596475468  0             0.107489   
    996       996   15366430552113981237 0             0.064617   
    998       998   12932831343316490139 0             0.087224   

                                                xiCMS_56_jes_up  \
run lumiblock event hash_id              slice                    
1   1         1     11792644136667482086 0             0.066901   
                                         0             0.066901   
                    6050478515959135830  0             0.023326   
                    3655724426672568106  0             0.019472   
                    13726604837472925229 0             0.065374   
...                                                         ...   
    995       995   12882894790533776945 0             0.018640   
                    8428429390990631370  0             0.140790   
                    6668014878596475468  0             0.024751   
    996       996   15366430552113981237 0             0.018321   
    998       998   12932831343316490139 0             0.027254   

                                                xiCMS_45_jes_dw  \
run lumiblock event hash_id              slice                    
1   1         1     11792644136667482086 0             0.046777   
                                         0             0.046777   
                    6050478515959135830  0             0.140454   
                    3655724426672568106  0             0.087528   
  

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.plot( df_signals_protons_multiRP_index[ label_ ][ "xi_nom" ], df_signals_protons_multiRP_index[ label_ ][ "sigma_xi" ], 'o' )

In [ ]:
msk_columns_eff_ = [ key_.find( 'eff' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_eff_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_eff_ ]
print ( columns_eff_ )
columns_ = [ "period" ]
columns_.extend( columns_eff_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_sf_ = [ key_.find( 'sf' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_sf_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_sf_ ]
columns_ = [ "period" ]
columns_.extend( columns_sf_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_proton_all_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_multitrack_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_strictzero_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_proton_all" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_multitrack" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_strictzero" ] ) )

In [ ]:
df_protons_multiRP_groupby = df_signals_protons_multiRP_index[ label_ ][ [ "arm" ] ].groupby( ["run","lumiblock","event","slice"] )
msk_2protons = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) >= 1 ) & ( np.sum( s_ == 1 ) >= 1 ) )
msk_1proton_Arm0 = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) >= 1 ) & ( np.sum( s_ == 1 ) == 0 ) )
msk_1proton_Arm1 = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) == 0 ) & ( np.sum( s_ == 1 ) >= 1 ) )
print ( msk_2protons )
print ( msk_1proton_Arm0 )
print ( msk_1proton_Arm1 )

In [ ]:
df_signals_protons_multiRP_index[ label_ ][ msk_2protons ]
# df_signals_protons_multiRP_index[ label_ ][ msk_1proton_Arm0 ]
# df_signals_protons_multiRP_index[ label_ ][ msk_1proton_Arm1 ]

In [ ]:
df_signals_protons_multiRP_events[ label_ ].columns

In [ ]:
df_signals_protons_multiRP_events[ label_ ]

In [ ]:
msk_columns_MX_ = [ key_.find( 'MX' ) >= 0 for key_ in df_signals_protons_multiRP_events[ label_ ].columns ]
columns_MX_ = df_signals_protons_multiRP_events[ label_ ].columns[ msk_columns_MX_ ]
columns_ = [ "period" ]
columns_.extend( columns_MX_ )
df_signals_protons_multiRP_events[ label_ ][ columns_ ]

In [ ]:
msk_columns_YX_ = [ key_.find( 'YX' ) >= 0 for key_ in df_signals_protons_multiRP_events[ label_ ].columns ]
columns_YX_ = df_signals_protons_multiRP_events[ label_ ].columns[ msk_columns_YX_ ]
columns_ = [ "period" ]
columns_.extend( columns_YX_ )
df_signals_protons_multiRP_events[ label_ ][ columns_ ]

In [ ]:
msk_columns_eff_ = [ key_.find( 'eff' ) >= 0 for key_ in df_signals_protons_multiRP_events[ label_ ].columns ]
columns_eff_ = df_signals_protons_multiRP_events[ label_ ].columns[ msk_columns_eff_ ]
print ( columns_eff_ )
columns_ = [ "period" ]
columns_.extend( columns_eff_ )
df_signals_protons_multiRP_events[ label_ ][ columns_ ]

In [ ]:
label_ = "GGToWW-AQGC-A0W1e-6"

msk_MWW = ( df_signals_protons_multiRP_events[ label_ ][ "MWW_nom" ] >= 500. )
print ( msk_MWW )

fig = plt.figure( figsize=(12,10) )

plt.hist( df_signals_protons_multiRP_events[ label_ ][ "jet0_pt" ][ msk_MWW ], histtype='step', color="darkblue", linestyle="-", bins=100, range=( 0., 1000. ) )

In [ ]:
label_ = "GGToWW-AQGC-A0W5e-6"
df_protons_multiRP_signal_index_ = df_signals_protons_multiRP_index[ label_ ]

msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_signal_index_[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_signal_index_[ "arm" ] == 1 )

print ( len(df_protons_multiRP_signal_index_[ "xi" ][ msk1 ]), len(df_protons_multiRP_signal_index_[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_signal_index_[ "trackx1" ][ msk1 ], df_protons_multiRP_signal_index_[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_signal_index_[ "trackx1" ][ msk2 ], df_protons_multiRP_signal_index_[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_signal_index_[ "trackx2" ][ msk1 ], df_protons_multiRP_signal_index_[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_signal_index_[ "trackx2" ][ msk2 ], df_protons_multiRP_signal_index_[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_signal_index_[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_signal_index_[ "arm" ] == 1 )

print ( len(df_protons_multiRP_signal_index_[ "xi" ][ msk1 ]), len(df_protons_multiRP_signal_index_[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_signal_index_[ "xi" ][ msk1 ], df_protons_multiRP_signal_index_[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_signal_index_[ "xi" ][ msk2 ], df_protons_multiRP_signal_index_[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )

### Background

In [ ]:
run_tables = True

resample_factor = 20
label = "data-random-resample_20"

fileNames_bkg = [
    "output-data-random-resample_20-2017B.h5",
    "output-data-random-resample_20-2017C.h5",
    "output-data-random-resample_20-2017D.h5",
    "output-data-random-resample_20-2017E.h5",
    "output-data-random-resample_20-2017F.h5"
]

import time
print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
time_s_ = time.time()

df_counts_bkg, df_protons_multiRP_bkg, df_protons_singleRP_bkg, df_ppstracks_bkg = 4 * [None]
df_protons_multiRP_bkg_index, df_protons_multiRP_bkg_events, df_ppstracks_bkg_index = 3 * [None]

if run_tables:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:
        df_counts_bkg_list_ = []
        df_protons_multiRP_bkg_index_list_ = []
        df_protons_multiRP_bkg_events_list_ = []
        df_ppstracks_bkg_index_list_ = []
        for file_ in fileNames_bkg:
            df_counts_bkg_, df_protons_multiRP_bkg_, df_protons_singleRP_bkg_, df_ppstracks_bkg_ = get_data( [ file_ ] )
            df_protons_multiRP_bkg_index_, df_protons_multiRP_bkg_events_, df_ppstracks_bkg_index_ = process_data_protons_multiRP( df_protons_multiRP_bkg_, df_ppstracks_bkg_, apply_fiducial=True, within_aperture=True, random_protons=True, runOnMC=False )
            df_counts_bkg_list_.append( df_counts_bkg_ )
            df_protons_multiRP_bkg_index_list_.append( df_protons_multiRP_bkg_index_ )
            df_protons_multiRP_bkg_events_list_.append( df_protons_multiRP_bkg_events_ )
            df_ppstracks_bkg_index_list_.append( df_ppstracks_bkg_index_ )
            
        df_counts_bkg_ = df_counts_bkg_list_[0]
        for idx_ in range( 1, len( df_counts_bkg_list_ ) ):
            df_counts_bkg_ = df_counts_bkg_.add( df_counts_bkg_list_[ idx_ ] )

        df_protons_multiRP_bkg_index_ = pd.concat( df_protons_multiRP_bkg_index_list_ )
        df_protons_multiRP_bkg_events_ = pd.concat( df_protons_multiRP_bkg_events_list_ )
        df_ppstracks_bkg_index_ = pd.concat( df_ppstracks_bkg_index_list_ )
        
        store_[ "counts" ] = df_counts_bkg_
        store_[ "protons_multiRP"] = df_protons_multiRP_bkg_index_
        store_[ "events_multiRP" ] = df_protons_multiRP_bkg_events_

with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    df_counts_bkg = store_[ "counts" ]
    df_protons_multiRP_bkg_index = store_[ "protons_multiRP" ]
    df_protons_multiRP_bkg_events = store_[ "events_multiRP" ]
        
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    print ( list( store_ ) )

In [ ]:
df_counts_bkg

In [ ]:
df_protons_multiRP_bkg_index

In [ ]:
df_protons_multiRP_bkg_events

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_bkg_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_bkg_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_bkg_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_bkg_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_bkg_index[ "trackx1" ][ msk1 ], df_protons_multiRP_bkg_index[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_bkg_index[ "trackx1" ][ msk2 ], df_protons_multiRP_bkg_index[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_bkg_index[ "trackx2" ][ msk1 ], df_protons_multiRP_bkg_index[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_bkg_index[ "trackx2" ][ msk2 ], df_protons_multiRP_bkg_index[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_bkg_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_bkg_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_bkg_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_bkg_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_bkg_index[ "xi" ][ msk1 ], df_protons_multiRP_bkg_index[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_bkg_index[ "xi" ][ msk2 ], df_protons_multiRP_bkg_index[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )

In [ ]:
labels_samples = {
    "GGToWW-AQGC-A0W1e-6": r"$\gamma\gamma \to WW \quad a_0^W=1e^{-6}$",
    "GGToWW-AQGC-A0W2e-6": r"$\gamma\gamma \to WW \quad a_0^W=2e^{-6}$",
    "GGToWW-AQGC-A0W5e-6": r"$\gamma\gamma \to WW \quad a_0^W=5e^{-6}$"
}

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "jet0_mass", bins=100, range=(0.,250.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 0 ] )
df_protons_multiRP_signal_events.hist( "jet0_mass", bins=100, range=(0.,250.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 0 ] )

df_protons_multiRP_bkg_events.hist( "jet0_corrmass", bins=100, range=(0.,250.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 1 ] )
df_protons_multiRP_signal_events.hist( "jet0_corrmass", bins=100, range=(0.,250.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 1 ] )
row_ += 1

df_protons_multiRP_bkg_events.hist( "num_bjets_ak8", bins=10, range=(0.,10.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 0 ] )
df_protons_multiRP_signal_events.hist( "num_bjets_ak8", bins=10, range=(0.,10.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 0 ] )

df_protons_multiRP_bkg_events.hist( "num_bjets_ak4", bins=10, range=(0.,10.), histtype="step", density=True, color="lightcoral", ax=axes[ row_,  1 ] )
df_protons_multiRP_signal_events.hist( "num_bjets_ak4", bins=10, range=(0.,10.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 1 ] )
row_ += 1


In [ ]:
msk_bkg = ( ( df_protons_multiRP_bkg_events.loc[ :, "jet0_corrmass"] >= 50.0 ) &
            ( df_protons_multiRP_bkg_events.loc[ :, "jet0_corrmass"] <= 110.0 ) & 
            ( df_protons_multiRP_bkg_events.loc[ :, "num_bjets_ak4"] == 0 ) )
msk_signal = ( ( df_protons_multiRP_signal_events.loc[ :, "jet0_corrmass"] >= 50.0 ) &
               ( df_protons_multiRP_signal_events.loc[ :, "jet0_corrmass"] <= 110.0 ) &
               ( df_protons_multiRP_signal_events.loc[ :, "num_bjets_ak4"] == 0 ) )
df_protons_multiRP_bkg_events = df_protons_multiRP_bkg_events.loc[ msk_bkg ]
df_protons_multiRP_signal_events = df_protons_multiRP_signal_events.loc[ msk_signal ]

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "jet0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "jet0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$p_{\rm{T}} (\rm{jet}) \,\rm{(GeV)}$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "jet0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "jet0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\eta (\rm{jet})$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

df_protons_multiRP_bkg_events.hist( "jet0_tau1", bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "jet0_tau1", bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$\tau_1 (\rm{jet})$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "jet0_tau2", bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "jet0_tau2", bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\tau_2 (\rm{jet})$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Jet.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Jet.png", bbox_inches='tight' )

#  'run', 'lumiblock', 'event', 'slice', 'xi', 'thx', 'thy', 't', 'ismultirp', 'rpid', 'arm', 
#  'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_energy', 'jet0_mass', 'jet0_corrmass', 'jet0_tau1', 'jet0_tau2', 'jet0_vertexz', 
#  'muon0_pt', 'muon0_eta', 'muon0_phi', 'muon0_energy', 'muon0_charge', 'muon0_iso', 'muon0_dxy', 'muon0_dz', 
#  'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 
#  'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi'

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "muon0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "muon0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$p_{\rm{T}} (\mu) \,\rm{(GeV)}$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "muon0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "muon0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\eta (\mu)$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

df_protons_multiRP_bkg_events.hist( "muon0_iso", bins=50, range=(0.,0.2), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "muon0_iso", bins=50, range=(0.,0.2), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$Isolation (\mu)$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "muon0_dz", bins=50, range=(0.,0.1), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "muon0_dz", bins=50, range=(0.,0.1), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$d_z (\mu) \,\rm{(cm)}$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Muon.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Muon.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "met", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "met", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$p_{\rm{T}}^{\rm{miss}} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "met_phi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "met_phi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$\phi({\vec p}_{\rm{T}}^{\rm{miss}})$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_MET.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_MET.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "nVertices", bins=100, range=(0.,100.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "nVertices", bins=100, range=(0.,100.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( "Number of vertices", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "pfcand_nextracks", bins=100, range=(0.,100.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "pfcand_nextracks", bins=100, range=(0.,100.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( "Number of extra tracks", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Vertices_ExtraTracks.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Vertices_ExtraTracks.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "WLeptonicPt", bins=100, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "WLeptonicPt", bins=100, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$p_{\rm{T}} (W^{\rm{lep}}) \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "WLeptonicPhi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "WLeptonicPhi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$\phi (W^{\rm{lep}})$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_WLeptonic.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_WLeptonic.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "recoMWW", bins=100, range=(0.,2000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "recoMWW", bins=100, range=(0.,2000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{WW} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "recoRapidityWW", bins=100, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "recoRapidityWW", bins=100, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{WW}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_WW.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_WW.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "MX", bins=100, range=(0.,2000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "MX", bins=100, range=(0.,2000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{\rm{pp}} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "YX", bins=100, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "YX", bins=100, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{\rm{pp}}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_protons.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_protons.png", bbox_inches='tight' )

In [ ]:
fig = plt.figure( figsize=(12,10) )

plt.hist( ( df_protons_multiRP_bkg_events[ "jet0_tau2" ] / df_protons_multiRP_bkg_events[ "jet0_tau1" ] ), bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral" )
plt.hist( ( df_protons_multiRP_signal_events[ "jet0_tau2" ] / df_protons_multiRP_signal_events[ "jet0_tau1" ] ), bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue" )
plt.xlabel( r"$\tau_{21}$", fontsize=20 )
plt.ylabel( "Density", fontsize=20 )
plt.title( "" )

if save_figures:
    plt.savefig( "plots/Variables_tau21.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_tau21.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )

df_protons_multiRP_bkg_events.hist( "shiftedRatioMWW_MX", bins=50, range=(-1.5,1.5), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "shiftedRatioMWW_MX", bins=50, range=(-1.5,1.5), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{WW} / M_{\rm{pp}} - 1$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{WW} - Y_{\rm{pp}}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_difference.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_difference.png", bbox_inches='tight' )

In [ ]:
msk_bkg_cut1 = ( np.abs( df_protons_multiRP_bkg_events[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_bkg_cut2 = ( np.abs( df_protons_multiRP_bkg_events[ "diffYWW_YX" ] ) <= 0.50 )
msk_bkg_A =  msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_B = ~msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_C =  msk_bkg_cut1 & ~msk_bkg_cut2
msk_bkg_D = ~msk_bkg_cut1 & ~msk_bkg_cut2

In [ ]:
x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(12,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='gray' )
plt.plot( (x_center,x_center), (y_min,y_max), color='gray' )
ax = plt.gca()
df_protons_multiRP_bkg_events[ msk_bkg_A ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightblue' )
df_protons_multiRP_bkg_events[ msk_bkg_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_bkg_events[ msk_bkg_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_bkg_events[ msk_bkg_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

n_events_bkg   = df_protons_multiRP_bkg_events.shape[0]
n_events_bkg_A = df_protons_multiRP_bkg_events[ msk_bkg_A ].shape[0]
n_events_bkg_B = df_protons_multiRP_bkg_events[ msk_bkg_B ].shape[0]
n_events_bkg_C = df_protons_multiRP_bkg_events[ msk_bkg_C ].shape[0]
n_events_bkg_D = df_protons_multiRP_bkg_events[ msk_bkg_D ].shape[0]
resample_factor = 20
print ( "Number of events: {}".format( n_events_bkg / resample_factor ) )
print ( "Number of events (A): {}".format( n_events_bkg_A / resample_factor ) )
print ( "Number of events (B): {}".format( n_events_bkg_B / resample_factor ) )
print ( "Number of events (C): {}".format( n_events_bkg_C / resample_factor ) )
print ( "Number of events (D): {}".format( n_events_bkg_D / resample_factor ) )

### Data

In [ ]:
# run_tables = True

# label = "data"

# fileNames_data = [
#     'output-data-2017B.h5',
#     'output-data-2017C.h5',
#     'output-data-2017D.h5',
#     'output-data-2017E.h5',
#     'output-data-2017F.h5'
# ]

# import time
# print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
# time_s_ = time.time()

# df_counts_data, df_protons_multiRP_data, df_protons_singleRP_data, df_ppstracks_data = 4 * [None]
# df_protons_multiRP_data_index, df_protons_multiRP_data_events, df_ppstracks_data_index =  3 * [None]

# if run_tables:
#     with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:

#         df_counts_data, df_protons_multiRP_data, df_protons_singleRP_data, df_ppstracks_data = get_data( fileNames_data )
#         df_protons_multiRP_data_index, df_protons_multiRP_data_events, df_ppstracks_data_index = process_data_protons_multiRP( df_protons_multiRP_data, df_ppstracks_data, apply_fiducial=True, within_aperture=True, runOnMC=False )

#         store_[ "counts" ] = df_counts_data
#         store_[ "protons_multiRP"] = df_protons_multiRP_data_index
#         store_[ "events_multiRP" ] = df_protons_multiRP_data_events

# with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
#     df_counts_data = store_[ "counts" ]
#     df_protons_multiRP_data_index = store_[ "protons_multiRP" ]
#     df_protons_multiRP_data_events = store_[ "events_multiRP" ]
        
# time_e_ = time.time()
# print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
# with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
#     print ( list( store_ ) )

In [ ]:
# label = "data"
label = "data-single-arm"
df_counts_data = None
df_protons_multiRP_data_index = None
df_protons_multiRP_data_events = None
with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    print ( list( store_ ) )
    df_counts_data = store_[ "counts" ]
    df_protons_multiRP_data_index = store_[ "protons_multiRP" ]
    df_protons_multiRP_data_events = store_[ "events_multiRP" ]

In [ ]:
df_counts_data

In [ ]:
df_protons_multiRP_data_index

In [ ]:
vars_ = [ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm", "random", "trackpixshift1", "rpid1", "trackpixshift2", "rpid2", "trackx1", "trackx2", "tracky1", "tracky2" ]
df_protons_multiRP_data_index[ vars_ ]

In [ ]:
df_protons_multiRP_data_events

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_data_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_data_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_data_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_data_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_data_index[ "trackx1" ][ msk1 ], df_protons_multiRP_data_index[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_data_index[ "trackx1" ][ msk2 ], df_protons_multiRP_data_index[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_data_index[ "trackx2" ][ msk1 ], df_protons_multiRP_data_index[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_data_index[ "trackx2" ][ msk2 ], df_protons_multiRP_data_index[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )
if save_figures:
    plt.savefig( "plots/Hit_Map_Data_vs_Sector_Near_Far.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Hit_Map_Data_vs_Sector_Near_Far.png", bbox_inches='tight' )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_data_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_data_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_data_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_data_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_data_index[ "xi" ][ msk1 ], df_protons_multiRP_data_index[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_data_index[ "xi" ][ msk2 ], df_protons_multiRP_data_index[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
if save_figures:
    plt.savefig( "plots/ThetaX_vs_Xi_Data_vs_Sector.pdf", bbox_inches='tight' )
    plt.savefig( "plots/ThetaX_vs_Xi_Data_vs_Sector.png", bbox_inches='tight' )

In [ ]:
msk_data = ( ( df_protons_multiRP_data_events.loc[ :, "jet0_corrmass"] >= 50.0 ) & ( df_protons_multiRP_data_events.loc[ :, "jet0_corrmass"] <= 110.0 ) )
df_protons_multiRP_data_events = df_protons_multiRP_data_events.loc[ msk_data ]

In [ ]:
msk_data_cut1 = ( np.abs( df_protons_multiRP_data_events[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_data_cut2 = ( np.abs( df_protons_multiRP_data_events[ "diffYWW_YX" ] ) <= 0.50 )
msk_data_A =  msk_data_cut1 &  msk_data_cut2
msk_data_B = ~msk_data_cut1 &  msk_data_cut2
msk_data_C =  msk_data_cut1 & ~msk_data_cut2
msk_data_D = ~msk_data_cut1 & ~msk_data_cut2
msk_data = ~msk_data_A

In [ ]:
x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(12,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='gray' )
plt.plot( (x_center,x_center), (y_min,y_max), color='gray' )
ax = plt.gca()
df_protons_multiRP_data_events[ msk_data_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_data_events[ msk_data_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_data_events[ msk_data_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

n_events_data_B = df_protons_multiRP_data_events[ msk_data_B ].shape[0]
n_events_data_C = df_protons_multiRP_data_events[ msk_data_C ].shape[0]
n_events_data_D = df_protons_multiRP_data_events[ msk_data_D ].shape[0]
resample_factor = 20
print ( "Number of events (B): {} - Ratio: {}".format( n_events_data_B, ( n_events_data_B / ( n_events_bkg_B / resample_factor ) ) ) )
print ( "Number of events (C): {} - Ratio: {}".format( n_events_data_C, ( n_events_data_C / ( n_events_bkg_C / resample_factor ) ) ) )
print ( "Number of events (D): {} - Ratio: {}".format( n_events_data_D, ( n_events_data_D / ( n_events_bkg_D / resample_factor ) ) ) )

### References